In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [2]:
from qiskit import IBMQ

# KQC Token
#TOKEN = "b5b31be2bb5237d52e8bdef143913cd62425eadcbda5c56c6f8ed8d067573ac0e07aa41d23d0bfcc880882a4142e2f812dad876fb3208d5c5f8f321be6626cce"
# Yonsei Token
TOKEN = "41bc5ff592064fefedbb4e5f949e42a9ded7835d2b14e69147aaab22b911ea16e1d95f6929e9b1fadf6c8e412253d50f820c25cd6c45be588d31910f2c4fd96b"


IBMQ.save_account(TOKEN, overwrite=True)
IBMQ.load_account() # Load account from disk


#KQC Provider
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='reservations')
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='kqc-research', project='kernel-experimen')

#Yonsei Provider
provider = IBMQ.get_provider(hub='ibm-q-yonsei', group='internal', project='reservations')




backend = "ibmq_kolkata"

dev = qml.device(
    'qiskit.ibmq',
    wires=27,
    shots=1024,
    backend=backend,
    provider=provider
)


PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_48.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_48.pt'


C:\Users\tak\AppData\Local\Temp\ipykernel_6852\50494149.py:9: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account(TOKEN, overwrite=True)
C:\Users\tak\AppData\Local\Temp\ipykernel_6852\50494149.py:9: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.save_account(TOKEN, overwrite=True)


In [3]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [4]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    

@qml.qnode(dev)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[12,15])
    embedding.U_TTN(param1, wires=[21,18])
    embedding.U_TTN(param2, wires=[15,18])
    return qml.expval(qml.PauliZ(18))

def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

# IBMQ_Jakarta


NQE trained on:          ibmq_toronto

QCNN trained on:         ibmq_jakarta

QCNN evaluated on:       ibmq_lagos

### Model 0

In [ ]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]


weights_Model0_Jakarta = [0.98390774, 0.14981093, 0.41799519, 0.82769965 ]
prediction_Model0_Jakarta = []
for x in X_test:
    prediction = Noisy_QCNN_classifier(weights_Model0_Jakarta, x, Trained=False)
    print(prediction)
    prediction_Model0_Jakarta.append(prediction)
accuracy_Model0_Jakarta = accuracy_test(prediction_Model0_Jakarta, Y_test)

In [ ]:
print(f"Accuracy with Model0_Jakarta pre-training: {accuracy_Model0_Jakarta}")

Accuracy: 0.528

### Model 2

In [ ]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
prediction_Model2_Jakarta = []
weights_Model2_Jakarta = [-0.01100251, 1.22128094, -1.86724874, 0.11109528 ]
for i in range(len(X_test)):
    x = X_test[i]
    prediction = Noisy_QCNN_classifier(weights_Model2_Jakarta, x, Trained='Model2')
    print(prediction)
    prediction_Model2_Jakarta.append(prediction)
accuracy_Model2_Jakarta = accuracy_test(prediction_Model2_Jakarta, Y_test)

In [ ]:
print(f"Accuracy with Model2_Jakarta pre-training: {accuracy_Model2_Jakarta}")

Accuracy: 0.964

### Model 3

In [ ]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]


weights_Model3_Jakarta = [2.01256055, 0.51212296, 2.11959629, -0.46627558]
prediction_Model3_Jakarta = [Noisy_QCNN_classifier(weights_Model3_Jakarta, x, Trained='Model3') for x in X_test]
accuracy_Model3_Jakarta = accuracy_test(prediction_Model3_Jakarta, Y_test)

In [ ]:
print(f"Accuracy with Model3_Jakarta pre-training: {accuracy_Model3_Jakarta}")

Accuracy: 0.92

# IBMQ_Toronto

NQE trained on: ibmq_toronto

QCNN trained on: ibmq_toronto

QCNN evaluated on: ibmq_kolkata

### Model 0

In [ ]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

prediction_Model0_Toronto = []
weights_Model0_Toronto = [ 0.37168039, 0.30243094, 0.74561994, 0.63088322]
for i in range(500):
    x = X_test[i]
    prediction = Noisy_QCNN_classifier(weights_Model0_Toronto, x, Trained=False)
    print(prediction)
    prediction_Model0_Toronto.append(prediction)
accuracy_Model0_Toronto = accuracy_test(prediction_Model0_Toronto, Y_test)

In [ ]:
print(f"Accuracy with Model0_Toronto pre-training: {accuracy_Model0_Toronto}")

### Model 2

In [ ]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]


In [ ]:
prediction_Model2_Toronto = []
weights_Model2_Toronto = [1.03388823, -0.14285616, 1.67144357, 1.13041186  ]
for i in range(21):
    x = X_test[i + 479]
    prediction = Noisy_QCNN_classifier(weights_Model2_Toronto, x, Trained='Model2')
    print(prediction)
    prediction_Model2_Toronto.append(prediction)
accuracy_Model2_Toronto = accuracy_test(prediction_Model2_Toronto, Y_test)

In [ ]:
print(f"Accuracy with Model2_Toronto pre-training: {accuracy_Model2_Toronto}")

In [ ]:
print(Y_test)

### Model3

In [5]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [7]:
prediction_Model3_Toronto = []
weights_Model3_Toronto = [1.15000724, -0.29707071, 1.56612213, -0.27187334 ]
for i in range(len(X_test)):
    x = X_test[i]
    prediction = Noisy_QCNN_classifier(weights_Model3_Toronto, x, Trained='Model3')
    print(prediction)
    prediction_Model3_Toronto.append(prediction)
accuracy_Model3_Toronto = accuracy_test(prediction_Model3_Toronto, Y_test)

C:\Users\tak\AppData\Local\Temp\ipykernel_6852\1893121398.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model3(torch.tensor(x))


tensor(0.2891, dtype=torch.float64)
tensor(-0.0352, dtype=torch.float64)
tensor(0.2852, dtype=torch.float64)
tensor(-0.0645, dtype=torch.float64)
tensor(-0.0469, dtype=torch.float64)
tensor(0.3535, dtype=torch.float64)
tensor(-0.1406, dtype=torch.float64)
tensor(0.0234, dtype=torch.float64)
tensor(0.3320, dtype=torch.float64)
tensor(0.3418, dtype=torch.float64)
tensor(0.3750, dtype=torch.float64)
tensor(0.3242, dtype=torch.float64)
tensor(0.3691, dtype=torch.float64)
tensor(0.5078, dtype=torch.float64)
tensor(0.0039, dtype=torch.float64)
tensor(0.3008, dtype=torch.float64)
tensor(-0.0332, dtype=torch.float64)
tensor(-0.1855, dtype=torch.float64)
tensor(0.3145, dtype=torch.float64)
tensor(0.3516, dtype=torch.float64)
tensor(0.4180, dtype=torch.float64)
tensor(0.3906, dtype=torch.float64)
tensor(-0.0586, dtype=torch.float64)
tensor(0.4785, dtype=torch.float64)
tensor(0.1211, dtype=torch.float64)
tensor(0.3027, dtype=torch.float64)
tensor(0.0684, dtype=torch.float64)
tensor(0.3789, dtype=

In [ ]:
print(f"Accuracy with Model3_Toronto pre-training: {accuracy_Model3_Toronto}")

# IBMQ Perth

NQE trained on: ibmq_toronto

QCNN trained on: ibmq_perth

QCNN evaluated on: ibmq_jakarta

### Model 0

In [ ]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

prediction_Model0_Perth = []
weights_Model0_Perth = [0.19449887, 0.82027649, 0.40379336, 0.18254816]
for i in range(500):
    x = X_test[i]
    prediction = Noisy_QCNN_classifier(weights_Model0_Perth, x, Trained=False)
    print(prediction)
    prediction_Model0_Perth.append(prediction)
accuracy_Model0_Perth = accuracy_test(prediction_Model0_Perth, Y_test)

### Model 2

In [ ]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
prediction_Model2_Perth = []
weights_Model2_Toronto = [1.25502256 -0.30062048 1.50211541 1.22376432]
for i in range(500):
    x = X_test[i]
    prediction = Noisy_QCNN_classifier(weights_Model2_Perth, x, Trained='Model2')
    print(prediction)
    prediction_Model2_Perth.append(prediction)
accuracy_Model2_Perth = accuracy_test(prediction_Model2_Perth, Y_test)

### Model 3

In [ ]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
prediction_Model3_Perth = []
weights_Model3_Toronto = [ ]
for i in range(500):
    x = X_test[i]
    prediction = Noisy_QCNN_classifier(weights_Model3_Perth, x, Trained='Model3')
    print(prediction)
    prediction_Model3_Perth.append(prediction)
accuracy_Model3_Perth = accuracy_test(prediction_Model3_Perth, Y_test)